# Análise de dados com SQL

### De onde vem o dataset?
O dataset usado é da competição hospedada no Kaggle [Predict Future Sales](https://www.kaggle.com/c/competitive-data-science-predict-future-sales/data). Consiste no histórico de vendas de algumas lojas na rússia entre Janeiro de 2013 e Outubro de 2015, provida pela [1C Company](https://1c.ru/eng/title.htm) para uma atividade no curso ["How to win a data science competition"](https://www.coursera.org/learn/competitive-data-science/home/welcome) na Coursera.

### Como está dividido esse documento?
Na primeira parte, o arquivo de banco de dados **SQLite** é criado no arquivo `sales.db`, e as tabelas construídas a partir da modelagem feita na imagem abaixo. A biblioteca Pandas é utilizada para processar os arquivos CSV e dividir os dados entre as colunas do banco.

![img](https://)

Na segunda parte o arquivo é alternativamente lido de um banco já preenchido e em seguida cada query é mostrada em funcionamento, conforme o jupyter notebook na raiz do repositório.

In [79]:
import os
from pathlib import Path
import sqlite3
import pandas as pd

def criar_banco(filename='sales.db'):
    os.remove(filename)
    Path('sales.db').touch()

def criar_tabelas(filename='sales.db'):
    conn = sqlite3.connect(filename)
    c = conn.cursor()


    c.execute('''
    CREATE TABLE item_categories (
        item_category_name VARCHAR(20),
        item_category_id INT(5),
        PRIMARY KEY (item_category_id)
    );
    ''')
    
    c.execute('''
    CREATE TABLE items (
        item_name VARCHAR(40) DEFAULT '',
        item_id INT(5),
        item_category_id INT(5),
        PRIMARY KEY (item_id)
    );
    ''')
    
    c.execute('''
    CREATE TABLE shops (
        shop_name VARCHAR(40) DEFAULT '',
        shop_id INT(5),
        PRIMARY KEY (shop_id)
    );
    ''')
    
    
    c.execute('''
    CREATE TABLE sales (
        sale_id INT(5) NOT NULL,
        item_id INT(5),
        date DATE(10),
        item_price DOUBLE(20),
        item_cnt_day INT(5),
        shop_id INT(5),
        PRIMARY KEY (sale_id)
    );
    ''')


    shops = pd.read_csv('shops.csv')
    shops.to_sql('shops', conn, if_exists='append', index = False)
    
    items = pd.read_csv('items.csv')
    items.to_sql('items', conn, if_exists='append', index = False)
    
    sales = pd.read_csv('sales_train.csv')
    sales['date'] = pd.to_datetime(sales['date'], format='%d.%m.%Y') # A data vem no formato russo, é necessário converter para ISO
    sales = sales.drop('date_block_num', 1) # Date block num é uma coluna redundante com a mesma informação que tem em `date`
    sales = sales.rename_axis('sale_id').reset_index() # SQLite3 não suporta autoincrement, então criamos uma coluna de id manualmente
    
    sales.to_sql('sales', conn, if_exists='append', index = False)
    
    item_categories = pd.read_csv('item_categories.csv')
    item_categories.to_sql('item_categories', conn, if_exists='append', index = False)
    conn.close()



In [80]:
arquivo_banco = 'sales.db'
criar_banco(arquivo_banco)
criar_tabelas(arquivo_banco)

conn = sqlite3.connect(arquivo_banco)
c = conn.cursor()

In [40]:
def ler_arquivo(arquivo):
    with open(arquivo, "r") as db:
        lines = list(db.readlines())

        return "\n".join(lines)

### Query 1 - Lista de jogos de PC mais vendidos em todas as lojas

In [43]:
query1 = ler_arquivo("query1.sql")
print(query1, end='\n\n\n')
pd.read_sql(query1, conn)

SELECT Sum(sl.item_cnt_day)                 AS "Vendas",

       it.item_name                         AS "Nome",

       Printf("$ %.2f", Avg(sl.item_price)) AS "Preço Médio",

       ct.item_category_name                AS "Categoria"

FROM   items it

       INNER JOIN item_categories ct

               ON it.item_category_id = ct.item_category_id

       INNER JOIN sales sl

               ON sl.item_id = it.item_id

WHERE  ct.item_category_name LIKE "Игры PC%"

GROUP  BY "nome"

ORDER  BY "vendas" DESC

LIMIT  50;




,Vendas,Nome,Preço Médio,Categoria
0,17245,"Diablo III [PC, Jewel, русская версия]",$ 999.58,Игры PC - Стандартные издания
1,10099,"Grand Theft Auto V [PC, русские субтитры]",$ 2051.76,Игры PC - Стандартные издания
2,10032,"Battlefield 4 [PC, русская версия]",$ 922.22,Игры PC - Стандартные издания
3,9012,World of Warcraft. Карта оплаты игрового време...,$ 808.54,Игры PC - Дополнительные издания
4,8262,"FIFA 14 [PC, русская версия]",$ 1018.78,Игры PC - Стандартные издания
5,7788,"Crysis 3 [PC, русская версия]",$ 636.41,Игры PC - Стандартные издания
6,7588,"Call of Duty: Black Ops II [PС, Jewel, русская...",$ 420.24,Игры PC - Стандартные издания
7,7351,StarCraft II: Heart of the Swarm (дополнение) ...,$ 644.06,Игры PC - Стандартные издания
8,7105,"Diablo III: Reaper of Souls (дополнение) [PC, ...",$ 888.23,Игры PC - Дополнительные издания
9,6975,"Battlefield 3 [PC, Jewel, русская версия]",$ 528.72,Игры PC - Стандартные издания


### Query 2 - Lista de lojas com mais de 5000 produtos anunciados e seu item mais vendido

In [44]:
query2 = ler_arquivo("query2.sql")
print(query2, end='\n\n\n')
pd.read_sql(query2, conn)

SELECT Count(it.item_name) AS "Itens a venda",

       sp.shop_name        AS "Nome da Loja",

       it.item_name        AS "Item mais vendido"

FROM   shops sp

       INNER JOIN sales sl

               ON sl.shop_id = sp.shop_id

       INNER JOIN items it

               ON it.item_id = sl.item_id

GROUP  BY it.item_name

HAVING "itens a venda" > 5000

ORDER  BY "itens a venda" DESC,

          Sum(sl.item_cnt_day)




,Itens a venda,Nome da Loja,Item mais vendido
0,31340,Москва Магазин С21,Фирменный пакет майка 1С Интерес белый (34*42)...
1,9408,"Москва ТРК ""Атриум""",Playstation Store пополнение бумажника: Карта ...
2,9067,Москва Магазин С21,Прием денежных средств для 1С-Онлайн
3,7479,"Москва ТРК ""Атриум""","Diablo III [PC, Jewel, русская версия]"
4,6853,"Москва ТРК ""Атриум""",Kaspersky Internet Security Multi-Device Russi...
5,6602,"Москва ТРК ""Атриум""",World of Warcraft. Карта оплаты игрового време...
6,6475,"Москва ТРК ""Атриум""","Grand Theft Auto V [PS3, русские субтитры]"
7,6320,"Москва ТРК ""Атриум""","Call of Duty: Black Ops II [PС, Jewel, русская..."
8,5811,"Москва ТРК ""Атриум""","Minecraft [Xbox 360, английская версия]"
9,5805,"Москва ТРК ""Атриум""","Grand Theft Auto V [Xbox 360, русские субтитры]"
